### KDT 5기
#### 1. Python 데이터분석: 예측모델 개발

- 강사: 최가람, 천재교육 AI센터 데이터기획분석팀원
- 주요 업무: 이탈 예측, 추천 (도서, 콘텐츠) 모델 개발 및 운영, 데이터 분석 업무

- 일정: 2023.11.02 ~ 11.03
<p>

- 교육내용
    - 이탈 예측 모델 개념 및 활용 데이터 설명
    - 데이터 전처리 모듈 개발
    - 분석 모델링 개발 실습 1 (Logistic Regression)
    - 분석 모델링 개발 실습 2 (LGBM)
    - 성능 지표 개념 (Confusion Matrix)
    - 모델 성능 결과 분석
    - 모델 최적화
<p>

- 교육목표
    - 온라인 교육 서비스를 이용하고 있는 유저들의 이력을 바탕으로 이탈 여부를 예측할 수 있는 모델을 개발해보자
    - 통합학년 이탈 예측 모델 구성
    - 학년 별 이탈 예측 모델 구성

- 과제: 이탈 예측 모델 개발
    - 모델의 전반적인 컨셉 -> 전학년 별 이탈 예측, 학년 별 이탈 예측, 성별에 따른 이탈 예측, 재구매 이력에 따른 이탈 예측 등
    - 활용 데이터 설명
    - 예측 모델(linear, logistic, DCT, Xgboost, LGBM 등등)
    - 성능 지표를 통한 결과 설명

#### 5. 최적화

##### 5-1 LGBM Optimization (Greedy Search, Bayesian Optimization)

In [2]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

In [6]:
# 데이터 읽어오기 (1학년 기준)
import pandas as pd
df = pd.read_csv("churn_data.csv")


In [4]:
del df['number']

In [15]:
# 학습 데이터, 예측 값 (label) 분리: 전체 학년 통합 (1~6학년) 이탈예측
# 'grade_sect_cd' 값 -> 1, 2, 3, 4, 5, 6 으로 치환 -> 'grade' 컬럼 구성

def change_value(values):
    if values == 'G011':
        return 1
    elif values == 'G012':
        return 2
    elif values == 'G013':
        return 3
    elif values == 'G014':
        return 4
    elif values == 'G015':
        return 5
    elif values == 'G016':
        return 6
    
df['grade'] = df['grade_sect_cd'].apply(change_value)
df[['grade']]
    



,grade
0,1
1,1
2,1
3,1
4,1
...,...
59995,6
59996,6
59997,6
59998,6


In [17]:
df1 = df.query('grade == 1')

In [18]:
df1

,number,system_id,grade_sect_cd,mbr_sex_cd,tmon_pchrg_lrn_dcnt,acmlt_pchrg_lrn_dcnt,acmlt_bilclct_amt,correct_rate_avg,learning_time_avg,media_action_cnt_sum,non_video_viewed_cnt_sum,get_mm_point_sum,label,re_purch,grade
0,3,87ee0282-ece5-4491-bb07-be93854750ac,G011,M,31.0,247.0,952000.0,0.59,670.48,673.0,306.0,1377.0,0,False,1
1,7,8817ada1-b7ad-4713-b041-ee180540cb87,G011,M,31.0,238.0,1071000.0,0.00,0.00,0.0,0.0,0.0,0,False,1
2,8,884cddf4-bda5-4d81-960c-b9a4892645f0,G011,F,31.0,92.0,436000.0,0.70,400.44,1067.0,468.0,6289.0,0,False,1
3,21,8a58832b-592b-4b33-b05e-0e11422d73d5,G011,M,31.0,370.0,1417000.0,0.81,914.47,840.0,212.0,1714.0,0,False,1
4,25,8af8f516-a9f0-4b6a-a27f-fc77ba51f708,G011,M,1.0,1.0,109000.0,0.00,0.00,0.0,0.0,0.0,0,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,82790,5c14dde5-33a4-4801-893b-cce7cc2ae82a,G011,M,31.0,226.0,952000.0,0.61,283.16,772.0,341.0,1602.0,0,False,1
9996,82813,5f9ccb34-fbdd-4544-9ade-2808d17ae904,G011,M,31.0,258.0,963900.0,0.87,1238.70,491.0,223.0,1001.0,0,False,1
9997,82827,621fb871-b2cd-48cc-abe0-62b57aa38eb1,G011,M,31.0,349.0,1308000.0,0.50,57.24,37008.0,2683.0,1188.0,0,False,1
9998,82831,632d0b31-a6ce-4cd9-89db-e34fc35278a8,G011,F,31.0,47.0,178000.0,0.95,199.05,1771.0,517.0,2316.0,0,False,1


In [21]:
# 데이터 준비 (X, y)
x = df1[['tmon_pchrg_lrn_dcnt',
       'acmlt_pchrg_lrn_dcnt',
       'acmlt_bilclct_amt',
       'correct_rate_avg', 
        'learning_time_avg',
        'media_action_cnt_sum',
        'non_video_viewed_cnt_sum']]
y = df1['label']
# 데이터를 학습용과 검증용으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
print(f"train 전체: {len(X_train)}, 이탈 : {y_train[y_train==1].shape[0]}, 미이탈: {y_train[y_train==0].shape[0]} ")
print(f"test 전체: {len(X_test)}, 이탈 : {y_test[y_test==1].shape[0]}, 미이탈: {y_test[y_test==0].shape[0]} ")

train 전체: 8000, 이탈 : 139, 미이탈: 7861 
test 전체: 2000, 이탈 : 35, 미이탈: 1965 


In [24]:
lgbm_model = LGBMClassifier(random_state = 0, verbose = 0)
lgbm_model.fit(X_train, y_train)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


LGBMClassifier(random_state=0, verbose=0)

In [26]:
# 기본 모델 성능 확인
# 예측 및 결과 출력
y_pred = lgbm_model.predict(X_test)
y_pred = list(y_pred)
print(f"예측 이탈 : {y_pred.count(1)}, 미이탈 :{y_pred.count(0)}")
# precision, recall, fscore
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, y_pred, average='binary')

예측 이탈 : 29, 미이탈 :1971


(0.896551724137931, 0.7428571428571429, 0.8125, None)

In [32]:
# LightGBM 모델 생성
model = LGBMClassifier(verbose = -1)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'num_leaves' : [10, 20, 30], 
    'max_depth' : [3, 5, 7],
    'learning_rate' : [0.01, 0.05, 0.1],
    'min_child_samples' : [10, 20, 30]
}

# GridSearchCV 객체 생성

grid_search = GridSearchCV(estimator=model, param_grid=param_grid)

# 그리드 서치 수행
grid_search.fit(X_train, y_train)



GridSearchCV(estimator=LGBMClassifier(verbose=-1),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'min_child_samples': [10, 20, 30],
                         'num_leaves': [10, 20, 30]})

In [33]:
# 최적의 하이퍼파라미터
print(grid_search.best_params_)
# 최적의 하이퍼파라미터로 학습된 모델 가져오기
best_model = grid_search.best_estimator_
# 예측 수행
y_pred = best_model.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='binary')

{'learning_rate': 0.1, 'max_depth': 7, 'min_child_samples': 30, 'num_leaves': 20}


(0.9615384615384616, 0.7142857142857143, 0.819672131147541, None)

In [51]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization

# Bayesian Optimization 범위 설정
param_bayesian = {
    'num_leaves' :(10, 20), 
    'max_depth' : (3, 7),
    'learning_rate' : (0.01, 0.1),
    'min_child_samples' : (10, 30)

}

# Objective Function 메소드 설정
def lgbm_objective(num_leaves, learning_rate, max_depth, min_child_samples):

    learning_rate = min(learning_rate, 0.01)
    params = {
        'num_leaves' : int(num_leaves), 
        'max_depth' : int(max_depth),
        'learning_rate' : learning_rate,
        'min_child_samples' : int(min_child_samples),
        'objective' : 'binary',
        'metric': 'binary_logloss' #평가지표 loss

    }

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    model = lgb.train(params, train_data, valid_sets=[valid_data])

    return model.best_score['valid_0']['binary_logloss']
    
# Bayesian Optimization 객체 생성
optimizer = BayesianOptimization(f=lgbm_objective, pbounds = param_bayesian)
# 최적화 수행
optimizer.maximize(init_points=10, n_iter=30)

# 최적의 하이퍼파라미터 출력
print(optimizer.max)
# 최적의 하이퍼파라미터로 LGBMClassifier 모델 학습
best_params = optimizer.max['params']
best_num_leaves = int(best_params['num_leaves'])
best_max_depth = int(best_params['max_depth'])
best_learning_rate = int(best_params['learning_rate'])
best_min_child_samples = int(best_params['min_child_samples'])

# 예측 수행
best_model = LGBMClassifier(num_leaves=best_num_leaves,
                            learning_rate=best_learning_rate,
                            max_depth=best_max_depth,
                            min_child_samples=best_min_child_samples,
                            objective='binary',
                            verbose = -1)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='binary')


|   iter    |  target   | learni... | max_depth | min_ch... | num_le... |
-------------------------------------------------------------------------
[LightGBM] [Info] Number of positive: 139, number of negative: 7861
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002011 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1348
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.017375 -> initscore=-4.035195
[LightGBM] [Info] Start training from score -4.035195
[1]	valid_0's binary_logloss: 0.0825346
[2]	valid_0's binary_logloss: 0.0790004
[3]	valid_0's binary_logloss: 0.0761871
[4]	valid_0's binary_logloss: 0.0738956
[5]	valid_0's binary_logloss: 0.0719523
[6]	valid_0's binary_logloss: 0.0703334
[7]	valid_0's binary_logloss: 0.069036
[8]	valid_0's binary_logloss: 0.0677944
[9]	valid_0's binary_logloss: 0.0666286
[

[LightGBM] [Fatal] Check failed: (learning_rate) > (0.0) at /Users/cbousseau/work/recipes/ci_py311/lightgbm_1677972953792/work/compile/src/io/config_auto.cpp, line 331 .



LightGBMError: Check failed: (learning_rate) > (0.0) at /Users/cbousseau/work/recipes/ci_py311/lightgbm_1677972953792/work/compile/src/io/config_auto.cpp, line 331 .


##### 5-2 [부록] Logisitc Regression cutoff 최적화

데이터셋 출처: https://www.kaggle.com/datasets/mdwasimakhtar03/social-network-adscsv

성능개선을 위한 cutoff 라인 변경: https://stackoverflow.com/questions/48882485/can-we-change-the-default-cut-off0-5-taken-by-logistic-regression-and-not-whil

In [ ]:
import pandas as pd
# 데이터 "Social_Network_Ads.csv" 읽어오기

In [ ]:
# 데이터 살펴보기

In [ ]:
# plot logistics curve
import seaborn as sns

In [ ]:
# 데이터 가져오기, Train, Test 데이터 분리하기
from sklearn.model_selection import train_test_split
# X : Age, EstimatedSalary
# Y : Purchased

In [ ]:
# 모델 학습
from sklearn.linear_model import LogisticRegression

In [ ]:
# Predict 값 도출

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 데이터 정규화

In [ ]:
# from sklearn.linear_model import LogisticRegression
# 모델 재훈련

# 재예측

# Confusion Matrix 재평가
from sklearn.metrics import confusion_matrix

In [ ]:
# cmtx 라벨 재구성

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# 최종 Yes(1) or No(0) 만 볼때:

# y = 0 또는 y = 1 일 "확률 "을 보고 싶을 때:

# y = 1 일 확률 값을 보고 싶을 때:

In [ ]:
# 기존의 Y_pred는 classifer.predict(X_test)를 통해, 0 또는 1을 리턴
# classifer.predict(X_test)는 모델의 예측한 y=1일 확률이 0.5보다 크면, 1로 리턴, 
# 0.5보다 작으면 0으로 리턴함 (default 로 설정되어 있기 때문)
# default 0.5 가 아닌 다른 cutoff 포인트를 고려해볼 수 있음

cutoff = 0.47 # 임의의 값 지정

In [ ]:
# 2. Logistic Regression 모델 cutoff 최적화에 적용
# 2.1 성능평가 연산 방식을 목적함수(objective function)로 정의
# 주의: minimize 함수에 적용을 위해 1-accuracy, 즉, 오류율 값으로 리턴. (Accuracy 기준의 목적함수 구성)

In [ ]:
#2.2 목적함수를 opt 모델에 적용

# 최적화 알고리즘

In [ ]:
# 2.3 cutoff 별 1-accuracy 값을 찍어서 어떻게 변화하는지 확인. (plt.scatter 이용)